In [13]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: Nelson Kang, Mane Muradyan, Phelan Niu
### Team Strategy Chosen: Market Beat

Goal: Market beat
Strategy: Sortino-centered stock selection with some short-term momentum tilt

Inputs:
- CSV file with tickers

Outputs:
- 

Game Plan:
1. load csv and set constants
2. get FX rate and static info of stocks
3. check eligibility of stocks (liquidity rule etc.)
4. Compute Sortino and momentum tilt and combine for a score
5. construct portfolio with cap and sector mix
6. Distribute stock weight


In [14]:
INITIAL_BUDGET_CAD = 1_000_000

# assignment rules
MIN_LIQ_AVG_VOLUME = 5000
MIN_STOCKS = 10
MAX_STOCKS = 25
MAX_SECTOR_WEIGHT = 0.40
MAX_STOCK_WEIGHT = 0.15

# lookback
MOM_LOOKBACK = 84
VOL_WINDOW = 60
CORR_WINDOW = 63
ATR_WINDOW = 42
TREND_HIGH_WINDOW = 63
TREND_LOW_WINDOW = 105

# for sharpe optimization
SHARPE_LOOKBACK = 126
SHARPE_TRIALS = 30000
RISK_FREE = 0.0

# date range
DATA_START_DATE = "2024-10-01"
DATA_END_DATE = "2025-11-21"

# fallback fx if usd to cad download fails
FALLBACK_USD_TO_CAD = 1.35

In [15]:
# LOAD TICKERS
def load_ticker_list(path):
    df = pd.read_csv(path)
    col = df.columns[0]
    vals = df[col].dropna().astype(str).str.upper().str.strip().tolist()
    header = str(col).upper().strip() # header might be ticker

    # prepend header
    if header not in vals:
        vals = [header] + vals

    return sorted(list(set(vals)))


# DOWNLOAD PRICES + META
def download_all_data(tickers, start, end):
    raw = yf.download(
        tickers, start=start, end=end,
        auto_adjust=False, group_by="ticker",
        progress=False, threads=True
    )

    if raw.empty:
        return (pd.DataFrame(),)*5

    # extract close, high, low, and volume
    if isinstance(raw.columns, pd.MultiIndex):
        # remove tickers that failed download
        valid = [t for t in tickers if t in raw.columns.levels[0]]
        close = pd.DataFrame({t: raw[t]["Close"] for t in valid})
        high = pd.DataFrame({t: raw[t]["High"] for t in valid})
        low = pd.DataFrame({t: raw[t]["Low"] for t in valid})
        vol = pd.DataFrame({t: raw[t]["Volume"] for t in valid})
    else:
        # single ticker fallback
        t = tickers[0]
        valid = [t]
        close = raw[['Close']].rename(columns={'Close': t})
        high = raw[['High']].rename(columns={'High': t})
        low = raw[['Low']].rename(columns={'Low': t})
        vol = raw[['Volume']].rename(columns={'Volume': t})

    # keep only tickers with at least one valid close price
    good = close.columns[close.notna().sum() > 0].tolist()
    close, high, low, vol = close[good], high[good], low[good], vol[good]

    rows = []
    for t in good:
        tk = yf.Ticker(t)
        try:
            info = tk.info
        except:
            info = {}

        sector = info.get("sector", "Unknown") or "Unknown"
        currency = info.get("currency", "USD") or "USD"
        mcap = info.get("marketCap", np.nan)
        country = info.get("country", None)
        exch = info.get("exchange", "")

        # assume if missing country
        if country is None:
            if currency == "CAD" or "TSX" in str(exch) or str(exch).endswith(".TO"):
                country = "CA"
            else:
                country = "US"

        rows.append({
            "Ticker": t,
            "Sector": sector,
            "Currency": currency,
            "MarketCap": mcap,
            "Country": country
        })

    meta = pd.DataFrame(rows)
    return close, high, low, vol, meta


# FILTER UNIVERSE
def filter_universe(meta, close, vol):
    if meta.empty:
        return meta

    # keep only US and CA stocks
    meta = meta[meta["Country"].isin(["US","CA"])].copy()

    # require min avg daily volume
    avg_vol = vol.mean()
    liquid = avg_vol[avg_vol >= MIN_LIQ_AVG_VOLUME].index
    meta = meta[meta["Ticker"].isin(liquid)].copy()

    # require enough price history for momentum window
    needed = MOM_LOOKBACK + 2
    hist_len = close.notna().sum()
    ok = hist_len[hist_len >= needed].index
    meta = meta[meta["Ticker"].isin(ok)].copy()

    if meta.empty:
        meta = meta.copy()

    return meta


In [16]:
# FACTORS
def compute_daily_returns(prices):
    if prices.empty:
        return pd.DataFrame()
    return prices.pct_change().dropna(how="all") # removes the empty first row


def compute_momentum(prices, lookback=MOM_LOOKBACK):
    recent = prices.iloc[-1]
    past = prices.iloc[-(lookback+1)]
    return recent / past - 1.0


def compute_volatility(returns, window=VOL_WINDOW):
    return returns.tail(window).std(axis=0)


def compute_avg_correlation(returns, window=CORR_WINDOW):
    corr = returns.tail(window).corr() # computes the correlation matrix
    return corr.apply(lambda c: (c.sum()-1)/(len(c)-1), axis=0) # computes the avg corr of each stock with all other stocks


def compute_atr(highs, lows, closes, window=ATR_WINDOW):
    prev_close = closes.shift(1) # true range formula uses previous closing price
    tr1 = highs - lows
    tr2 = (highs - prev_close).abs()
    tr3 = (lows - prev_close).abs()

    tr = pd.concat([tr1, tr2, tr3], axis=0).groupby(level=0).max()

    atr = tr.rolling(window).mean()
    return atr.iloc[-1] # today's ATR


def compute_trend_signal(closes, atr, high_window=TREND_HIGH_WINDOW, low_window=TREND_LOW_WINDOW):
    rolling_high = closes.rolling(high_window).max().iloc[-1] # highest closing price
    rolling_low  = closes.rolling(low_window).min().iloc[-1] # lowest closing price
    cur = closes.iloc[-1] # today's closing price

    upper = rolling_high + atr
    lower = rolling_low - atr

    trend = pd.Series(0, index=closes.columns) # default neutral
    trend[cur > upper] = 1 # confirmed uptrend
    trend[cur < lower] = -1 # confirmed downtrend
    return trend


def score_calc(meta, prices, highs, lows):
    if meta.empty:
        return meta.copy()

    tickers = meta["Ticker"].tolist()
    close_sub = prices[tickers]
    high_sub = highs[tickers]
    low_sub = lows[tickers]

    # compute returns
    returns = compute_daily_returns(close_sub)
    if returns.empty:
        return meta.iloc[:0]

    # compute factors, fail-safe on exceptions
    try:
        M = compute_momentum(close_sub)
        V = compute_volatility(returns)
        C = compute_avg_correlation(returns)
        ATR = compute_atr(high_sub, low_sub, close_sub)
        T = compute_trend_signal(close_sub, ATR)
    except:
        return meta.iloc[:0]

    # convert factors to ranks
    rank_M = M.rank(ascending=False)
    rank_V = V.rank(ascending=True)
    rank_C = C.rank(ascending=True)
    rank_T = T.rank(ascending=False)

    # final weighted RAAM score
    score = 0.40*rank_M + 0.30*rank_V + 0.25*rank_C + 0.05*rank_T

    # pack into DataFrame
    factors = pd.DataFrame({
        "Ticker": tickers,
        "Momentum": M.values,
        "Volatility": V.values,
        "AvgCorr": C.values,
        "Trend": T.values,
        "Score": score.values
    }).set_index("Ticker")

    return meta.set_index("Ticker").join(factors, how="inner").reset_index()


In [17]:
# SELECT STOCKS
def select_top_stocks(meta_scored):
    if meta_scored.empty:
        return meta_scored.copy()

    df = meta_scored.sort_values("Score").copy() # lower score = better rank

    chosen = []
    sector_w = {} # sector weight accumulator

    for _, row in df.iterrows():
        t = row["Ticker"]
        s = row["Sector"]

        tmp = chosen + [t]
        if len(tmp) > MAX_STOCKS:
            break

        w = 1.0 / len(tmp)
        sw = sector_w.copy()
        sw[s] = sw.get(s, 0) + w

        # reject if sector overweight
        if max(sw.values()) > MAX_SECTOR_WEIGHT:
            continue

        chosen.append(t)
        sector_w = sw

        # early stop if min count reached
        if len(chosen) >= MIN_STOCKS:
            break

    # fallback if not enough picked then take first N
    if len(chosen) < MIN_STOCKS:
        chosen = df["Ticker"].head(MIN_STOCKS).tolist()

    return df[df["Ticker"].isin(chosen)].copy()


# SHARPE OPTIMIZATION
def optimize_sharpe(close, meta_sel):
    tickers = meta_sel["Ticker"].tolist()
    n = len(tickers)
    if n == 0:
        return pd.Series(dtype=float)

    # returns for sharpe window
    rets = close[tickers].pct_change().dropna().tail(SHARPE_LOOKBACK)
    if rets.empty:
        return pd.Series([1/n]*n, index=tickers)

    mu = rets.mean() # mean return vector
    sigma = rets.cov() # covariance matrix

    best_w, best_sh = None, -999
    rng = np.random.default_rng(42)

    # random search with constraints
    for _ in range(SHARPE_TRIALS):
        w = rng.dirichlet(np.ones(n)) # random weights
        if (w > MAX_STOCK_WEIGHT).any(): # per stock cap
            continue
        if (w < (1/(2*n))).any(): # minimum floor
            continue

        volp = np.sqrt(np.dot(w, sigma @ w)) # portfolio volatility
        if volp == 0:
            continue

        sh = np.dot(w, mu) / volp # sharpe ratio
        if sh > best_sh:
            best_w, best_sh = w, sh

    # fallback is equal weights
    if best_w is None:
        best_w = np.array([1/n]*n)

    return pd.Series(best_w, index=tickers)

# BUILD PORTFOLIO
def build_portfolio(close, weights, meta_sel):
    tickers = weights.index.tolist()
    last = close.iloc[-1][tickers] # last price for each ticker

    # get usd to cad FX rate
    try:
        fx = yf.download("USDCAD=X", period="5d", progress=False)["Close"].iloc[-1]
    # fallback if api fails
    except:
        fx = FALLBACK_USD_TO_CAD

    rows = []

    for t in tickers:
        row = meta_sel[meta_sel["Ticker"] == t].iloc[0]

        cur = str(row["Currency"]).upper()
        price = float(last[t]) # ensure float

        # convert price to cad
        price_cad = float(price * fx) if cur == "USD" else float(price)

        val_cad = float(weights[t]) * INITIAL_BUDGET_CAD
        
        shares = val_cad / price_cad if price_cad > 0 else 0.0

        rows.append({
            "Ticker": t,
            "Sector": row["Sector"],
            "Currency": cur,
            "Price": price,
            "Shares": shares,
            "Weight": float(weights[t]),
            "Value": val_cad
        })

    return pd.DataFrame(rows)


In [18]:
# FINAL IMPLEMENTATION
def run_raam_simple(path):
    # load tickers
    tickers = load_ticker_list(path)

    # download prices and metadata
    close, high, low, vol, meta = download_all_data(
        tickers, DATA_START_DATE, DATA_END_DATE
    )

    # if nothing usable
    if close.empty or meta.empty:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # filter universe
    meta2 = filter_universe(meta, close, vol)

    # fallback if filtering removed everything
    if meta2.empty:
        meta2 = meta[meta["Ticker"].isin(close.columns)].copy()

    # score using raam
    meta_scored = score_calc(meta2, close, high, low)

    # fallback if no score due to insufficient history
    if meta_scored.empty:
        tickers_ok = meta2["Ticker"].tolist()
        prices_ok = close[tickers_ok].dropna(axis=1, how="all")

        if prices_ok.empty:
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

        # use shorter momentum window if data is very short
        lb = min(MOM_LOOKBACK, len(prices_ok) - 1)
        mom_simple = prices_ok.iloc[-1] / prices_ok.iloc[-(lb+1)] - 1

        meta_scored = meta2.copy()
        meta_scored["Momentum"] = meta_scored["Ticker"].map(mom_simple)
        meta_scored["Volatility"] = np.nan
        meta_scored["AvgCorr"] = np.nan
        meta_scored["Trend"] = 0
        # negative momentum ranked low -> better score
        meta_scored["Score"] = (-meta_scored["Momentum"]).rank()

    # sector caps
    meta_sel = select_top_stocks(meta_scored)

    # sharpe ratio optimization
    weights = optimize_sharpe(close, meta_sel)

    # build final portfolio
    portfolio = build_portfolio(close, weights, meta_sel)

    return portfolio, meta_scored, meta_sel


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Nelson Kang, Mane Muradyan, Phelan Niu


In [20]:
portfolio_final, meta_scored, selected_stocks = run_raam_simple("Tickers_file.csv")

display(selected_stocks)
display(portfolio_final)


4 Failed downloads:
['MON', 'CELG', 'AGN', 'RTN']: YFTzMissingError('possibly delisted; no timezone found')
/var/folders/0g/ymzkzh_x5jldygb71vv6z4t00000gp/T/ipykernel_53228/2505461896.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return prices.pct_change().dropna(how="all") # removes the empty first row
/var/folders/0g/ymzkzh_x5jldygb71vv6z4t00000gp/T/ipykernel_53228/1079696252.py:49: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = close[tickers].pct_change().dropna().tail(SHARPE_LOOKBACK)
/var/folders/0g/ymzkzh_x5jldygb71vv6z4t00000gp/T/ipykernel_53228/1079696252.py:88: Future

,Ticker,Sector,Currency,MarketCap,Country,Momentum,Volatility,AvgCorr,Trend,Score
28,RY.TO,Financial Services,CAD,298110615552,Canada,0.154995,0.006315,0.155676,0,9.80
19,LMT,Industrials,USD,107576025088,United States,0.113447,0.010990,0.052921,0,9.95
17,KO,Consumer Defensive,USD,313952501760,United States,0.030535,0.009944,0.109063,0,11.30
15,CAT,Industrials,USD,258875473920,United States,0.271489,0.022512,-0.000976,0,11.90
0,AAPL,Technology,USD,4029017227264,United States,0.245556,0.014515,0.148224,0,12.05
22,PEP,Consumer Defensive,USD,200337080320,United States,0.011003,0.012958,0.036407,0,12.75
18,LLY,Healthcare,USD,949975187456,United States,0.295321,0.019457,0.100614,0,12.85
31,TD.TO,Financial Services,CAD,199101218816,Canada,0.122687,0.010426,0.193858,0,13.30
1,ABBV,Healthcare,USD,417597620224,United States,0.202379,0.016246,0.145425,0,13.90
34,UNP,Industrials,USD,134184861696,United States,0.003129,0.009823,0.147868,0,15.00


,Ticker,Sector,Currency,Price,Shares,Weight,Value
0,RY.TO,Financial Services,CAD,207.979996,404.206022,0.084067,84066.766685
1,LMT,Industrials,USD,468.260010,191.956865,0.126712,126711.907788
2,KO,Consumer Defensive,USD,71.209999,941.929997,0.094555,94555.396240
3,CAT,Industrials,USD,546.130005,165.463939,0.127387,127387.292368
4,AAPL,Technology,USD,266.250000,317.583011,0.119199,119199.268304
5,PEP,Consumer Defensive,USD,146.100006,427.429652,0.088032,88032.207384
6,LLY,Healthcare,USD,1043.290039,100.437666,0.147716,147716.286642
7,TD.TO,Financial Services,CAD,114.660004,571.615862,0.065541,65541.476802
8,ABBV,Healthcare,USD,229.449997,166.119590,0.053732,53732.323059
9,UNP,Industrials,USD,221.210007,298.413122,0.093057,93057.074727
